In [1]:
import pandas as pd
import numpy as np
import gc
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.cross_validation import KFold, StratifiedKFold, cross_val_score
from datetime import datetime
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler,LabelEncoder, OneHotEncoder, minmax_scale, scale,PolynomialFeatures
from sklearn import tree
from sklearn import linear_model
from sklearn import svm
from sklearn import neighbors
from sklearn import ensemble
from sklearn.feature_selection import SelectFromModel, VarianceThreshold,RFE
from minepy import MINE
from mlxtend.regressor import StackingRegressor
from collections import defaultdict

c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data_set1 = pd.read_excel('复赛数据/训练_20180117.xlsx')
data_set2 = pd.read_excel('复赛数据/测试A_20180117.xlsx')
y2 = pd.read_csv('复赛数据/[new] fusai_answer_a_20180127.csv', names=['ID', 'Value'])


submit = pd.read_csv('复赛数据/answer_sample_b_20180117.csv', names=['id', 'Y'])
test_set = pd.read_excel('复赛数据/测试B_20180117.xlsx')

data_set2 = pd.concat([data_set2, y2[['Value']]], axis=1)
data_set = pd.concat([data_set1, data_set2], axis=0)


y = data_set[['Value']]
data_set = data_set.drop(['Value'], axis=1)
Le = LabelEncoder()

In [3]:
input_data = pd.concat([data_set, test_set], ignore_index=True)
input_data = input_data.drop(['ID'], axis=1)

In [4]:
col = pd.DataFrame(input_data.ix[:, 2:].columns, columns=['col'])
TOOL_ID_col_dict = defaultdict(lambda : 0)
for index,row in col.iterrows():
    info=row.col.split('X')[0]
    TOOL_ID_col_dict[info] += 1

TOOL_ID_col_dict

defaultdict(<function __main__.<lambda>>,
            {'210': 228,
             '220': 179,
             '300': 21,
             '310': 170,
             '311': 184,
             '312': 679,
             '330': 889,
             '340': 139,
             '344': 398,
             '360': 1030,
             '400': 183,
             '420': 173,
             '440A': 213,
             '520': 382,
             '750': 1030,
             'Chamber': 1,
             'Chamber ID': 1,
             'ERROR:#N/A': 1,
             'ERROR:#N/A (#1)': 1,
             'ERROR:#N/A (#2)': 1,
             'ERROR:#N/A (#3)': 1,
             'ERROR:#N/A_1': 1,
             'ERROR:#N/A_1 (#1)': 1,
             'ERROR:#N/A_1 (#2)': 1,
             'ERROR:#N/A_10': 1,
             'ERROR:#N/A_11': 1,
             'ERROR:#N/A_12': 1,
             'ERROR:#N/A_13': 1,
             'ERROR:#N/A_14': 1,
             'ERROR:#N/A_15': 1,
             'ERROR:#N/A_16': 1,
             'ERROR:#N/A_17': 1,
             'ERROR

In [5]:
#删除全空字段
null_col = []
for col in data_set.columns:
    if data_set[col].isnull().all() == True:
        null_col.append(col)
        
input_data.drop(null_col, axis=1, inplace=True)    
input_data.drop(['220X150', '220X151'], axis=1, inplace=True)    

In [6]:
#转换obj字段
obj_col = []
for col in input_data.columns:
    if input_data[col].dtypes == object:
        obj_col.append(col)
        
for i in range(len(obj_col)):
    Le.fit(input_data[obj_col[i]].unique())
    input_data[obj_col[i]] = Le.transform(input_data[obj_col[i]])

In [7]:
#删除较大值
def date_cols(data):
    for col in data:
        if data[col].min() > 1e13:
            data = data.drop([col], axis=1)   
    return data

input_data = date_cols(input_data)

In [8]:
#drop相同列
def drop_col(data):
    data = data.fillna(data.mean())      
    for line in data.columns:
        if len(data[line].unique()) == 1:
            data = data.drop([line], axis=1)    
    return data

input_data = drop_col(input_data)

In [9]:
#四分位数处理异常值
def quantile_dropout(data):  
    for c in data.columns:
        Q1 = data[c].quantile(q=0.25, interpolation='linear')
        Q3 = data[c].quantile(q=0.75, interpolation='linear')   
    
        min_v = Q1 - 3 * (Q3 - Q1)
        max_v = Q3 + 3 * (Q3 - Q1)
    
        data[c][(data[c] >= max_v) | (data[c] <= min_v)] = data[c].mean()
           
    return data

#异常值处理完再清晰一遍数据
input_data = quantile_dropout(input_data)
input_data.fillna(input_data.mean(), inplace=True)
input_data = drop_col(input_data)

c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [10]:
#剔除小方差字段
VT=VarianceThreshold(threshold=0.5)
input_data = pd.DataFrame(VT.fit_transform(input_data))

In [32]:
train = input_data.ix[:1099, :]
test = input_data.ix[1100:, :]
clf_gt = ensemble.GradientBoostingRegressor(learning_rate=0.01, n_estimators=200, random_state=0)
clf_gt.fit(train, y)

model = SelectFromModel(clf_gt, prefit=True) 
train = pd.DataFrame(model.transform(train))
test = pd.DataFrame(model.transform(test))

In [18]:
y.reset_index(inplace=True)
y.drop(['index'], axis=1,inplace=True)
y

,Value
0,2.814025
1,2.782955
2,2.725973
3,2.794768
4,2.787539
5,2.569175
6,2.965824
7,2.965824
8,2.773921
9,2.773921


In [29]:
train.to_csv('train111.csv', index=False)
test.to_csv('test111.csv', index=False)
y.to_csv('y111.csv', index=False)

In [43]:
#线下
def wmaeEval(preds, dtrain):
    label = dtrain.get_label()
    return 'error', np.sum(np.square(preds - label)) / len(label)
param = {}
param['eta'] = 0.01
param['max_depth'] = 3
param['subsample'] = 0.8
param['colsample_bytree'] = 0.6
param['seed'] = 0
xgbTrain = xgb.DMatrix(train, label=y)
modle = xgb.cv(param, xgbTrain, num_boost_round=9800, feval=wmaeEval, nfold=5)
print(modle.iloc[-1, 0]) 

0.0105672


In [ ]:
0.0105672


In [41]:
gbdt = ensemble.GradientBoostingRegressor(n_estimators=5200,loss='huber',learning_rate=0.01,subsample=0.8,min_samples_leaf=5,max_features='sqrt')
cv_model = cross_val_score(gbdt, train, y,  cv=5, scoring='neg_mean_squared_error')
(np.mean(np.abs(cv_model)))

c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\sklearn\utils\validation.py:526: DataC

0.02216024959481629

In [44]:
#线上
param = {}
param['eta'] = 0.01
param['max_depth'] = 3
param['subsample'] = 0.8
param['colsample_bytree'] = 0.6
param['seed'] = 0
#S_sub = []
#for feature_num in [210,200,230,190,220,240]:
#    train = input_data.ix[:1099, :]
#    test = input_data.ix[1100:, :]
#    clf_gt = ensemble.GradientBoostingRegressor(learning_rate=0.01, n_estimators=feature_num, random_state=0)
#    clf_gt.fit(train, y)

#    model = SelectFromModel(clf_gt, prefit=True) 
#    train = pd.DataFrame(model.transform(train))
#    test = pd.DataFrame(model.transform(test))
xgbTrain = xgb.DMatrix(train, label=y)
xgbTest = xgb.DMatrix(test)
modle = xgb.train(param, xgbTrain, num_boost_round=9200)
    
submit['Y'] = (modle.predict(xgbTest))

In [48]:
submit['Y'] = fin['mean'].values 

In [49]:
submit

,id,Y
0,NH0464,2.901397
1,NH0465,2.986080
2,NH0466,2.674103
3,NH0467,2.783338
4,NH0468,2.635935
5,NH0469,2.754563
6,NH0470,2.640132
7,NH0471,2.525705
8,NH0472,3.006060
9,NH0473,2.656115


In [50]:
submit.describe()

,Y
count,412.000000
mean,2.765557
std,0.169875
min,2.401403
25%,2.638269
50%,2.750970
75%,2.878292
max,3.274439


In [44]:
submit

,id,Y
0,NH0464,2.878818
1,NH0465,2.919634
2,NH0466,2.634657
3,NH0467,2.750538
4,NH0468,2.602232
5,NH0469,2.710890
6,NH0470,2.611614
7,NH0471,2.466172
8,NH0472,2.933717
9,NH0473,2.613623


In [51]:
submit.to_csv('一发入魂{}.csv'.format(datetime.now().strftime('%Y%m%d_%H%M%S')), index=False, header=None)

In [20]:
model_gb = ensemble.GradientBoostingRegressor(n_estimators=450, 
                                              max_depth=2, 
                                              subsample=0.8, 
                                              learning_rate=0.01, 
                                              random_state=2, 
                                              max_features=0.2)
modle0  = xgb.XGBRegressor(learning_rate=0.01, 
                           max_depth=3, 
                           colsample_bytree=0.6, 
                           subsample=0.8, 
                           seed=2, 
                           n_estimators=8800)
modle1  = xgb.XGBRegressor(learning_rate=0.01, 
                           max_depth=3, 
                           colsample_bytree=0.6, 
                           subsample=0.8, 
                           seed=0, 
                           n_estimators=8800)

clf1 = lgb.LGBMRegressor(colsample_bytree=0.3,
                         learning_rate=0.01, 
                         subsample=0.8, 
                         num_leaves=4, 
                         objective='regression', 
                         n_estimators=1200, 
                         seed=2)
base_model = [['xgb0', modle0],
              ['xgb1', modle1], 
              ['gb', model_gb],
              ['lgb', clf1],]


folds = list(KFold(len(train), n_folds=5, random_state=0))
S_train = np.zeros((train.shape[0], len(base_model)))
S_test = np.zeros((test.shape[0], len(base_model)))    
for index, item in enumerate(base_model):
    print("the model", index)
    clf = item[1]
    S_test_i = np.zeros((test.shape[0], len(folds)))
    for j, (train_idx, test_idx) in enumerate(folds):
        X_train = train.ix[train_idx, :]
        X_valid = train.ix[test_idx, :]
        Y = y.ix[train_idx, :]
        clf.fit(X_train, Y['Value'])
        S_train[test_idx, index] = clf.predict(X_valid)
        S_test_i[:, j] = clf.predict(test)       
    S_test[:, index] = S_test_i.mean(1)
    
print('Done')

the model 0
the model 1
the model 2
the model 3
Done


In [23]:
linreg1 = linear_model.LinearRegression()
linreg1.fit(S_train, y)

result = linreg1.predict(S_test)
result

array([[ 2.91280695],
       [ 3.00728619],
       [ 2.68588218],
       [ 2.80692562],
       [ 2.65203793],
       [ 2.78700961],
       [ 2.65256158],
       [ 2.5881346 ],
       [ 3.03188003],
       [ 2.66678301],
       [ 2.79704475],
       [ 2.82130765],
       [ 2.52779458],
       [ 2.73871225],
       [ 2.75843558],
       [ 2.94481697],
       [ 2.67314272],
       [ 2.92576991],
       [ 2.87012021],
       [ 3.26826838],
       [ 2.84924604],
       [ 3.08088731],
       [ 2.61872774],
       [ 2.77210176],
       [ 2.77124776],
       [ 2.99137954],
       [ 3.03609138],
       [ 2.9533413 ],
       [ 2.84841885],
       [ 2.72539836],
       [ 2.82514963],
       [ 2.68721196],
       [ 2.63808745],
       [ 2.97820777],
       [ 2.54795255],
       [ 2.88308472],
       [ 2.78374093],
       [ 2.57493598],
       [ 3.13905597],
       [ 2.79486808],
       [ 2.82538388],
       [ 2.55010282],
       [ 2.76804154],
       [ 3.00925288],
       [ 2.82193156],
       [ 2

In [25]:
pd.DataFrame(result).describe()

,0
count,412.000000
mean,2.777924
std,0.171946
min,2.388597
25%,2.655355
50%,2.766907
75%,2.893097
max,3.298111


In [26]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression,ridge,Lasso
from sklearn.model_selection import train_test_split 
from sklearn import cross_validation
from sklearn.metrics import mean_squared_error   
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler 
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier,GradientBoostingRegressor 
from sklearn.svm import SVR
from sklearn.preprocessing import LabelEncoder
import warnings 
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings('ignore')

def select_col_feature(train_,test_,Y_train,col_coef_=0.1):
  train_Temp = pd.concat((train_,Y_train),axis = 1)
  train_corr = train_Temp.corr()
  strong_feature = []
  train_columns_=train_Temp.columns
  for col in train_columns_:
     crr = np.abs(train_corr.loc['Value',col]) 
     if crr >= col_coef_ and col != 'Value':
        strong_feature.append(col)
  train_1=train_[strong_feature]
  test_1=test_[strong_feature]
  print('poson_col:',train_1.shape)
  return train_1,test_1


def optim_data(train,test_A,del_cols):
    train_cols = [x for x in train.columns if x not in del_cols]
    test_cols = [x for x in train_cols if x!='Value']
    return train[train_cols],test_A[test_cols]

def train_Processing(train,test):
    train_new = pd.DataFrame({'index':train.index})
    test_new = pd.DataFrame({'index':test.index})
    train_y = pd.DataFrame({'Value':train['Value']})
    for col in train.columns:
        if col == 'Value':
                continue
        if train[col].dtype == 'object':
                lb = LabelEncoder()
                lb.fit(pd.concat([train[col],test[col]],axis=0))
                train_new[col] = lb.transform(train[col])
                test_new[col] = lb.transform(test[col])
        else:
                train_test = pd.concat([train[col],test[col]],axis=0)
                train_new[col] = (train[col] - np.min(train_test)) / (1e-6 + np.max(train_test) - np.min(train_test))
                test_new[col] = (test[col] - np.min(train_test)) / (1e-6 + np.max(train_test) - np.min(train_test))
       
    train_new.drop('index',axis=1,inplace=True)
    test_new.drop('index',axis=1,inplace=True)
    return train_new.fillna(train_new.mean()),test_new.fillna(test_new.mean()),train_y  

train = data_set
test_A = test_set
test_ID=test_A.ID

same_cols = []
for col in train.columns:
    if train[col].nunique() <= 1:
        same_cols.append(col)

train,test_A = optim_data(train,test_A,same_cols)

null_cols = []
for col in test_A.columns:
    if test_A[col].nunique() < 1:
        null_cols.append(col)
train,test_A = optim_data(train,test_A,null_cols)

train_corr = train.corr()
#quchu chongfu
del_same_cols = []
cols = train_corr.columns
for i in range(len(cols)):
    col1 = cols[i]
    if col1 in del_same_cols:
        continue
    for j in range(i+1,len(cols)):
        col2 = cols[j]
        if train_corr.loc[col1,col2] == 1:
            if train.loc[0,col1] == train.loc[0,col2]:
                del_same_cols.append(col2)

train,test_A = optim_data(train,test_A,del_same_cols)

train.drop('ID',axis=1,inplace=True)
test_A.drop('ID',axis=1,inplace=True)


print("train.shape",train.shape)

train_new,test_new,train_y = train_Processing(train,test_A)

model = GradientBoostingRegressor(n_estimators=2500,loss='huber',learning_rate=0.02,subsample=0.6,min_samples_leaf=5,max_features='sqrt')
model.fit(train_new,train_y)

model_randomforests = RandomForestRegressor(n_estimators=1000,max_features='sqrt',min_samples_leaf=5)
model_randomforests.fit(train_new,train_y)




def get_top_features(feature,model,topN):
    feature_importance = pd.DataFrame({'feature':feature,'importance':model.feature_importances_})
    feature_importance = feature_importance.sort_values('importance',ascending=False)
    return feature_importance['feature'][:topN]

train = train.drop([702])
train_y = train_y.drop([702])
train_new = train_new.drop([702])
print("train_new.shape",train_new.shape)



sub = pd.DataFrame()
sub['ID'] = test_ID

sub_A=0

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [46]:
fin = pd.DataFrame(result, columns=['stacking'])
fin['singal'] = modle.predict(xgbTest)
fin['mean'] = fin.mean(axis=1)
fin

,stacking,singal,mean
0,2.912807,2.889987,2.901397
1,3.007286,2.964874,2.986080
2,2.685882,2.662324,2.674103
3,2.806926,2.759750,2.783338
4,2.652038,2.619831,2.635935
5,2.787010,2.722117,2.754563
6,2.652562,2.627702,2.640132
7,2.588135,2.463275,2.525705
8,3.031880,2.980239,3.006060
9,2.666783,2.645448,2.656115


In [47]:
fin.describe()

,stacking,singal,mean
count,412.000000,412.000000,412.000000
mean,2.777924,2.753191,2.765557
std,0.171946,0.168770,0.169875
min,2.388597,2.410836,2.401403
25%,2.655355,2.627594,2.638269
50%,2.766907,2.732697,2.750970
75%,2.893097,2.866366,2.878292
max,3.298111,3.250767,3.274439
